# Advanced Pulsar Actuator Control via PCP_over_USB in Python (Logging to CSV)
This script is designed to connect to a Pulsar actuator via a CAN-over-USB adapter, log sensor data at high frequency, and save it to a CSV file.

In [ ]:
# Import necessary modules
from pcp_api.PulsarActuator import PulsarActuator
from pcp_api.can_over_usb import PCP_over_USB
from time import sleep, time
import csv

## Connect to the CAN Adapter

In [ ]:
# Automatically detect the CAN port
port = PCP_over_USB.get_port()
print(f"Connecting to {port}")

# Initialize the adapter
adapter = PCP_over_USB(port)

# Initialize the Actuator

This creates an actuator object at address 0

In [ ]:
actuator = PulsarActuator(adapter, 0)

## Specify the Logging Variables
Specifies which sensor data to log (e.g., torque sensor, PCB temperature)

In [ ]:
itemsToLog = [
    PulsarActuator.PCP_Items.TORQUE_SENS_RAW,
    PulsarActuator.PCP_Items.TEMP_PCB,
]

## Connect to the Actuator

In [ ]:
if not actuator.connect():
    print(f"Could not connect to the actuator {actuator.address}")
    adapter.close()
    exit(1)
print(f"Connected to the actuator {actuator.address}")

## Configuration of the Feedback Rates
* Sets up high-frequency feedback (1kHz) for selected items.
* Disables low-frequency feedback.

In [ ]:
actuator.setHighFreqFeedbackItems(itemsToLog)
actuator.setHighFreqFeedbackRate(actuator.Rates.RATE_1KHZ)
actuator.setLowFreqFeedbackRate(actuator.Rates.DISABLED)

## Preaparig CSV Logging 
This opens a CSV file for writing

In [ ]:
file = open("log.csv", "w")
csv_writer = csv.writer(file, lineterminator='\n')

Adding a hearer row with the timestamp and item names

In [ ]:
# add header
header = [time()]  # timestamp
header.extend([item.name for item in sorted(itemsToLog, key=lambda x: x.name)])  # sorted by name for consistent order
csv_writer.writerow(header)

## Define Feedback Callback
This function is called whenever new feedback data is recieved

In [ ]:
def actuator_feedback(address: int, feedback: dict):
    line = [time()]  # timestamp
    line.extend([feedback[k] for k in sorted(feedback.keys())])  # sorted keys for consistent order
    csv_writer.writerow(line)

## Start Actuator

In [ ]:
actuator.change_mode(PulsarActuator.Mode.SPEED)
actuator.change_setpoint(1)
actuator.start()
actuator.set_feedback_callback(actuator_feedback)

## Run and Cleanup 
This runs for 3 seconds, then disconnects and closes everything

In [ ]:
try:
    sleep(3)  # actuator_feedback() should be triggered during this time
except KeyboardInterrupt:
    pass
finally:
    actuator.disconnect()  # also stops the actuator
    sleep(0.1)
    adapter.close()
    file.close()


The CSV is updated for postprocessing